In [56]:
import pandas as pd
import datetime

In [57]:
pd.options.display.max_columns = None
pd.options.display.expand_frame_repr =False

Read in the raw data using pandas

In [71]:
raw_yield_data = pd.read_csv('../data_raw/raw_yield_data.csv')
df=raw_yield_data.copy()
df = df.drop(columns=["Unnamed: 0"])



Remove the units from the attributes and only keep the numerical value

In [72]:
df['Mileage'] = df['Mileage'].str.extract(r'(\d+\.?\d*)').astype(float)
df['Engine']  = df['Engine'].str.extract(r'(\d+\.?\d*)').astype(float)
df['Power']   = df['Power'].str.extract(r'(\d+\.?\d*)').astype(float)

Check missing values count by column 

In [73]:
df.isnull().sum()

Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64

Drop the New_Price column because more than 83% of the column values are missing

In [74]:
df = df.drop(columns=["New_Price"])

Fill missing values only when a model has a median, otherwise use global median

In [76]:
df['Mileage'] = df['Mileage'].fillna(df['Mileage'].median())

df['Engine'] = (
    df.groupby('Name')['Engine']
      .transform(lambda x: x.fillna(x.median()))
      .fillna(df['Engine'].median())
)

df['Power'] = (
    df.groupby('Name')['Power']
      .transform(lambda x: x.fillna(x.median()))
      .fillna(df['Power'].median())
)

df['Seats'] = (
    df.groupby('Name')['Seats']
      .transform(lambda x: x.fillna(x.median()))
      .fillna(df['Seats'].median())
)
df.isnull().sum()

Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Price                0
dtype: int64

 Change the categorical variables (“Fuel_Type” and “Transmission”) into numerical one hot encoded value

In [77]:
df = pd.get_dummies(df, columns=['Fuel_Type', 'Transmission'])

Add Car_Age feature

In [78]:
current_year = datetime.datetime.now().year
df['Car_Age'] = current_year - df['Year']

Perform data wrangling

In [80]:
df = df.rename(columns={'Mileage': 'Mileage_kmpl', 'Engine': 'Engine_CC', 'Power': 'Power_bhp'})


In [83]:
df.head(10)

,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage_kmpl,Engine_CC,Power_bhp,Seats,Price,Fuel_Type_Diesel,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual,Car_Age
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,12.50,True,False,False,False,True,10
1,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,4.50,False,False,True,False,True,14
2,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,6.00,True,False,False,False,True,13
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,17.74,True,False,False,True,False,12
4,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,3.50,True,False,False,False,True,12
5,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,First,11.36,2755.0,171.50,8.0,17.50,True,False,False,True,False,9
6,Volkswagen Vento Diesel Comfortline,Pune,2013,64430,First,20.54,1598.0,103.60,5.0,5.20,True,False,False,False,True,12
7,Tata Indica Vista Quadrajet LS,Chennai,2012,65932,Second,22.30,1248.0,74.00,5.0,1.95,True,False,False,False,True,13
8,Maruti Ciaz Zeta,Kochi,2018,25692,First,21.56,1462.0,103.25,5.0,9.95,False,False,True,False,True,7
9,Honda City 1.5 V AT Sunroof,Kolkata,2012,60000,First,16.80,1497.0,116.30,5.0,4.49,False,False,True,True,False,13


Write clean dataframe to disk

In [85]:
df.to_csv('../data_clean/clean_yield_data.csv',index=False)